# Movie Hit Prediction System

## Project Overview
This is a production-ready machine learning system designed to predict movie success through a two-stage approach:
1. **Regression Stage**: Predict movie revenue before release
2. **Classification Stage**: Categorize movies as Flop / Average / Hit

## Phase 1: Data Loading and Consolidation

This phase covers:
- Loading both TMDB datasets
- Schema inspection and comparison
- Duplicate detection and removal
- Data standardization
- Consolidated dataset creation

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

PROJECT_ROOT = Path("/home/asif/AI-Movie-Hit-Predictor")
DATA_DIR = PROJECT_ROOT / "project_components" / "data"
DATASET_PATH_5K = DATA_DIR / "02_raw_tmdb_5000_movies.csv"
DATASET_PATH_10K = DATA_DIR / "01_raw_tmdb_10000_movies.csv"

df_5k = pd.read_csv(DATASET_PATH_5K)
df_10k = pd.read_csv(DATASET_PATH_10K)

print(f"\n✓ Dataset 1 (02_raw_tmdb_5000_movies.csv) loaded: {df_5k.shape[0]} rows × {df_5k.shape[1]} columns")
print(f"✓ Dataset 2 (01_raw_tmdb_10000_movies.csv) loaded: {df_10k.shape[0]} rows × {df_10k.shape[1]} columns")
print(f"\nTotal rows before consolidation: {df_5k.shape[0] + df_10k.shape[0]}")


✓ Dataset 1 (02_raw_tmdb_5000_movies.csv) loaded: 4803 rows × 20 columns
✓ Dataset 2 (01_raw_tmdb_10000_movies.csv) loaded: 10000 rows × 9 columns

Total rows before consolidation: 14803


### Step 1.1: Dataset Comparison

**Dataset 1 (5K)**: TMDB's well-curated dataset with comprehensive features
- Budget and Revenue data
- Production companies and keywords
- Detailed metadata

**Dataset 2 (10K)**: Extended TMDB dataset with more movies
- May have partial overlap with 5K dataset
- Smaller feature set (missing budget/revenue)
- Useful for expanding coverage

In [2]:
print("\nDataset 1 (5K) Columns")
print(f"Columns: {sorted(df_5k.columns.tolist())}")
print(f"Data Types:\n{df_5k.dtypes}")

print("\n--- Dataset 2 (10K) Columns ---")
print(f"Columns: {sorted(df_10k.columns.tolist())}")
print(f"Data Types:\n{df_10k.dtypes}")

# Find common and unique columns
cols_5k = set(df_5k.columns)
cols_10k = set(df_10k.columns)
common_cols = cols_5k & cols_10k
unique_to_5k = cols_5k - cols_10k
unique_to_10k = cols_10k - cols_5k

print(f"\nSchema Analysis")
print(f"✓ Common columns ({len(common_cols)}): {sorted(common_cols)}")
print(f"✓ Unique to 5K dataset ({len(unique_to_5k)}): {sorted(unique_to_5k)}")
print(f"✓ Unique to 10K dataset ({len(unique_to_10k)}): {sorted(unique_to_10k)}")


Dataset 1 (5K) Columns
Columns: ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']
Data Types:
budget                    int64
genres                      str
homepage                    str
id                        int64
keywords                    str
original_language           str
original_title              str
overview                    str
popularity              float64
production_companies        str
production_countries        str
release_date                str
revenue                   int64
runtime                 float64
spoken_languages            str
status                      str
tagline                     str
title                       str
vote_average            float64
vote_count                int64
dtype: object

--- Dataset 2 (10K

In [3]:
# Missing values analysis
print("\n--- Missing Values Analysis (Dataset 1 - 5K) ---")
missing_5k = df_5k.isnull().sum()
missing_5k_pct = (missing_5k / len(df_5k) * 100).round(2)
missing_5k_df = pd.DataFrame({
    'Column': missing_5k.index,
    'Missing Count': missing_5k.values,
    'Missing %': missing_5k_pct.values
}).sort_values('Missing Count', ascending=False)
print(missing_5k_df[missing_5k_df['Missing Count'] > 0].to_string(index=False))

print("\n--- Missing Values Analysis (Dataset 2 - 10K) ---")
missing_10k = df_10k.isnull().sum()
missing_10k_pct = (missing_10k / len(df_10k) * 100).round(2)
missing_10k_df = pd.DataFrame({
    'Column': missing_10k.index,
    'Missing Count': missing_10k.values,
    'Missing %': missing_10k_pct.values
}).sort_values('Missing Count', ascending=False)
print(missing_10k_df[missing_10k_df['Missing Count'] > 0].to_string(index=False))


--- Missing Values Analysis (Dataset 1 - 5K) ---
      Column  Missing Count  Missing %
    homepage           3091      64.36
     tagline            844      17.57
    overview              3       0.06
     runtime              2       0.04
release_date              1       0.02

--- Missing Values Analysis (Dataset 2 - 10K) ---
  Column  Missing Count  Missing %
overview              6       0.06


### Step 2.1: Schema Summary

**Key Insights:**
- Both datasets share core columns: `id`, `title`, `original_language`, `original_title`, `overview`, `popularity`, `release_date`, `vote_average`, `vote_count`
- Dataset 1 (5K) has financial data: `budget`, `revenue` (critical for our prediction task)
- Dataset 1 (5K) has rich metadata: `genres`, `keywords`, `production_companies`, `production_countries`, `spoken_languages`
- Dataset 2 (10K) appears to be a superset that may contain the same movies as 5K
- Missing values are significant in both datasets - need careful imputation strategy

In [4]:
print("\n--- Duplicate Detection by TMDB ID ---")

# Check duplicates within each dataset
dup_5k_id = df_5k[df_5k.duplicated(subset=['id'], keep=False)]
dup_10k_id = df_10k[df_10k.duplicated(subset=['id'], keep=False)]

print(f"Duplicates in 5K dataset (by id): {len(dup_5k_id)} records")
print(f"Duplicates in 10K dataset (by id): {len(dup_10k_id)} records")

# Remove intra-dataset duplicates (keep first occurrence)
df_5k_dedup = df_5k.drop_duplicates(subset=['id'], keep='first')
df_10k_dedup = df_10k.drop_duplicates(subset=['id'], keep='first')

print(f"\n✓ After removing intra-dataset duplicates:")
print(f"  Dataset 1: {df_5k.shape[0]} → {df_5k_dedup.shape[0]} rows")
print(f"  Dataset 2: {df_10k.shape[0]} → {df_10k_dedup.shape[0]} rows")

# Check for inter-dataset overlap (movies present in both datasets)
ids_5k = set(df_5k_dedup['id'].values)
ids_10k = set(df_10k_dedup['id'].values)
overlap_ids = ids_5k & ids_10k

print(f"\n--- Inter-Dataset Overlap ---")
print(f"✓ Movies in both datasets: {len(overlap_ids)}")
print(f"✓ Movies only in 5K: {len(ids_5k - ids_10k)}")
print(f"✓ Movies only in 10K: {len(ids_10k - ids_5k)}")


--- Duplicate Detection by TMDB ID ---
Duplicates in 5K dataset (by id): 0 records
Duplicates in 10K dataset (by id): 0 records

✓ After removing intra-dataset duplicates:
  Dataset 1: 4803 → 4803 rows
  Dataset 2: 10000 → 10000 rows

--- Inter-Dataset Overlap ---
✓ Movies in both datasets: 3297
✓ Movies only in 5K: 1506
✓ Movies only in 10K: 6703


### Step 3.1: Deduplication Strategy

**Approach:**
1. **Primary Key**: Use TMDB `id` field (globally unique identifier)
2. **Intra-dataset deduplication**: Remove duplicates within each dataset
3. **Inter-dataset merge**: Keep all unique movies from both datasets
4. **Preference**: For overlapping movies, prefer Dataset 1 (5K) as it has richer financial data

In [5]:
def standardize_dataframe(df, dataset_name):
   
    df = df.copy()
    
    print(f"\n--- Standardizing {dataset_name} ---")
    
    # 1. Standardize release_date to datetime
    if 'release_date' in df.columns:
        df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
        print(f"✓ Converted 'release_date' to datetime")
    
    # 2. Standardize numeric columns
    numeric_cols = ['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"✓ Standardized numeric columns: {[c for c in numeric_cols if c in df.columns]}")
    
    # 3. Ensure string columns are lowercase (for consistency in categorical data)
    string_cols = df.select_dtypes(include=['object']).columns
    for col in string_cols:
        if col not in ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']:
            # Only lowercase simple text fields, not JSON-like data
            df[col] = df[col].fillna('').astype(str)
    
    # 4. Drop rows with missing critical identifiers
    if 'id' in df.columns:
        initial_rows = len(df)
        df = df.dropna(subset=['id'])
        dropped = initial_rows - len(df)
        if dropped > 0:
            print(f"✓ Dropped {dropped} rows with missing 'id'")
    
    return df

# Standardize both datasets
df_5k_std = standardize_dataframe(df_5k_dedup, "Dataset 1 (5K)")
df_10k_std = standardize_dataframe(df_10k_dedup, "Dataset 2 (10K)")


--- Standardizing Dataset 1 (5K) ---
✓ Converted 'release_date' to datetime
✓ Standardized numeric columns: ['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count']

--- Standardizing Dataset 2 (10K) ---
✓ Converted 'release_date' to datetime
✓ Standardized numeric columns: ['popularity', 'vote_average', 'vote_count']


/tmp/ipykernel_443/3367850433.py:20: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  string_cols = df.select_dtypes(include=['object']).columns
/tmp/ipykernel_443/3367850433.py:20: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes 

### Step 4.1: Data Standardization Details

**Transformations Applied:**
1. **Date fields**: Convert to `datetime64` format for time-series analysis
2. **Numeric fields**: Convert to float/int with proper handling of missing values
3. **String fields**: Ensure consistent formatting (handle nulls, whitespace)
4. **Critical fields**: Remove rows with missing TMDB IDs (primary key)

In [6]:
# Get IDs from standardized datasets
ids_5k_std = set(df_5k_std['id'].values)
ids_10k_std = set(df_10k_std['id'].values)

# Find movies only in 10K
unique_to_10k_ids = ids_10k_std - ids_5k_std
df_10k_unique = df_10k_std[df_10k_std['id'].isin(unique_to_10k_ids)]

print(f"\n--- Merge Strategy ---")
print(f"✓ Base dataset (5K): {len(df_5k_std)} movies")
print(f"✓ Additional movies from 10K: {len(df_10k_unique)} movies")
print(f"✓ Expected consolidated dataset: {len(df_5k_std) + len(df_10k_unique)} movies")

all_cols = set(df_5k_std.columns) | set(df_10k_unique.columns)
print(f"\n✓ Total unique columns in merged dataset: {len(all_cols)}")

# Perform the merge: reindex df_10k_unique to match df_5k_std columns, then add any extra columns
cols_to_add = [col for col in df_10k_unique.columns if col not in df_5k_std.columns]
df_10k_unique_reindexed = df_10k_unique.reindex(columns=df_5k_std.columns, fill_value=np.nan)

df_consolidated = pd.concat([
    df_5k_std,
    df_10k_unique_reindexed
], ignore_index=True, sort=False)

print(f"\n--- Consolidation Results ---")
print(f"✓ Final consolidated dataset shape: {df_consolidated.shape[0]} rows × {df_consolidated.shape[1]} columns")
print(f"✓ Columns in consolidated dataset: {sorted(df_consolidated.columns.tolist())}")


--- Merge Strategy ---
✓ Base dataset (5K): 4803 movies
✓ Additional movies from 10K: 6703 movies
✓ Expected consolidated dataset: 11506 movies

✓ Total unique columns in merged dataset: 20

--- Consolidation Results ---
✓ Final consolidated dataset shape: 11506 rows × 20 columns
✓ Columns in consolidated dataset: ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']


In [7]:
# Verify the consolidation
print("\n--- Data Quality Check ---")
print(f"✓ Unique movies (by id): {df_consolidated['id'].nunique()}")
print(f"✓ Duplicate IDs remaining: {len(df_consolidated[df_consolidated.duplicated(subset=['id'], keep=False)])}")

# Verify no data loss
if df_consolidated['id'].nunique() == len(df_consolidated):
    print("✓ ✓ ✓ NO DUPLICATES - Consolidation successful!")
else:
    print("⚠ WARNING: Duplicates detected after consolidation!")
    
# Data availability summary
print("\n--- Feature Availability in Consolidated Dataset ---")
data_available = {
    'Total movies': len(df_consolidated),
    'With budget data': df_consolidated['budget'].notna().sum() if 'budget' in df_consolidated.columns else 0,
    'With revenue data': df_consolidated['revenue'].notna().sum() if 'revenue' in df_consolidated.columns else 0,
    'With both budget & revenue': (df_consolidated['budget'].notna() & df_consolidated['revenue'].notna()).sum() if 'budget' in df_consolidated.columns and 'revenue' in df_consolidated.columns else 0,
}
for key, value in data_available.items():
    pct = (value / data_available['Total movies'] * 100) if key != 'Total movies' else 0
    if key != 'Total movies':
        print(f"  {key}: {value} ({pct:.1f}%)")
    else:
        print(f"  {key}: {value}")


--- Data Quality Check ---
✓ Unique movies (by id): 11506
✓ Duplicate IDs remaining: 0
✓ ✓ ✓ NO DUPLICATES - Consolidation successful!

--- Feature Availability in Consolidated Dataset ---
  Total movies: 11506
  With budget data: 4803 (41.7%)
  With revenue data: 4803 (41.7%)
  With both budget & revenue: 4803 (41.7%)


### Step 5.1: Consolidated Dataset Summary

The consolidation process:
1. **Preserves** all movies from the 5K dataset (prioritized for data richness)
2. **Adds** unique movies from the 10K dataset that weren't in 5K
3. **Maintains** all available columns from both datasets
4. **Ensures** no data duplication (verified by TMDB ID uniqueness)

In [8]:
output_path = DATA_DIR / "03_consolidated_movies.csv"
df_consolidated.to_csv(output_path, index=False)
print(f"\n✓ Consolidated dataset saved to: {output_path}")

# Generate summary statistics
print("\n--- Consolidated Dataset Statistics ---")
print(f"Shape: {df_consolidated.shape}")
print(f"\nData Types:")
print(df_consolidated.dtypes)

print(f"\n\nFirst 5 rows (sample):")
print(df_consolidated[['id', 'title', 'budget', 'revenue', 'release_date', 'popularity']].head())

print(f"\n\nDescriptive Statistics (numeric columns):")
print(df_consolidated.describe())


✓ Consolidated dataset saved to: /home/asif/AI-Movie-Hit-Predictor/project_components/data/03_consolidated_movies.csv

--- Consolidated Dataset Statistics ---
Shape: (11506, 20)

Data Types:
budget                         float64
genres                          object
homepage                        object
id                               int64
keywords                        object
original_language                  str
original_title                     str
overview                           str
popularity                     float64
production_companies            object
production_countries            object
release_date            datetime64[us]
revenue                        float64
runtime                        float64
spoken_languages                object
status                          object
tagline                         object
title                              str
vote_average                   float64
vote_count                       int64
dtype: object


First 5 rows